### When should you NOT use a list comprehension? <br>
There are two common cases where you shouldn't use a list comprehension: <br>
- you don't actually want a list
- the logic is too long <br>
***Case 1***: You don't actually want a list <br>
List comprehensions build lists, but that's not the only reason we use for loops. Sometimes we have functions or methods whose main purpose is their side effect, and they don't return anything meaningful. 

[team.set_location(HOME) for team in league_teams if team in home_teams_today]

Not only does this look more natural as a conventional for loop, it doesn't waste space creating a list that it just ignores. 

In [1]:
for team in league_teams: 
    
    if team in home_teams_today: 
        team.set_location(HOME)

NameError: name 'league_teams' is not defined

***Case 2***: The logic is too long <br>
One of the main benefits of list comprehensions is that they make your code shoert and clearer. Once you start packing too much into a single statement, it becomes harder to follow than a regular for loop. <br>
For example <br>



In [2]:
active_player_accounts = [player.get_account() for team in league_teams
                          if len(team.roster) >1 for player in team.get_players()
                          if not player.is_injured()]

NameError: name 'league_teams' is not defined

While descriptive variable names went along way into making this piece of code somewhat readable, it's still hard to understand. 


In [3]:
active_player_account = []

for team in league_teams: 
    #teams need to have at least 2 players before they're considered 'active'
    if len(team.roster) <=1: 
        continue
    
    for player in team.get_players():
        
        #only want active players
        if player.is_injured():
            continue
        account = player.get_account()
        active_player_accounts.append(account)

NameError: name 'league_teams' is not defined

The above is clearer because we can see: 
- it includes comments to explain the code
- use control flow keywords like continue
- debug this section of code more easily using logging statements or asserts
- easily see the complexity of the code by scanning down the lines, picking out for loops

### Slicing lists from the end
We want to run some analystics on our investments. To start, we're given a list contianing the balance at the end of the day for some number of days. The last item in the list represents yesterday's closing balance and each previous item refer to the day before. 

In [4]:
daily_balances = [107.92, 108.67, 109.86, 110.15]

The first step in the process is to grab adjacent items in the list. To get started, we want a function that takes in our list of daily_balances and prints pairs of adjacent balances for the last 3 days:

In [5]:
show_balances(daily_balances)

NameError: name 'show_balances' is not defined

***Should print: ***<br>
"slice starting 3 days ago: [108.67, 109.86]" <br>
"slice starting 2 days ago: [109.86, 110.15]"

We just hired a new intern, Dan, to help us with this but something doesn't seem to be working quite right. here's his function: 
    

In [6]:
def show_balances(daily_balances): 
    
    #do not include -1 because that slice will only have 1 balance, yesterday
    for day in range(-3, -1):
        balance_slice = daily_balances[day : day +2]
        
        #use positive number for printing
        print("slice starting %d days ago: %s" % (abs(day)), balance_slice)

What's his code printing, and how can we fix it?

In [7]:
#let's run his code and see what it prints
show_balances(daily_balances)

TypeError: not enough arguments for format string

In [8]:
#this is what we're supposed to get from the above code of Dan's
"slice starting 3 days ago: [108.67, 109.86]"
"slice starting 2 days ago: []"

'slice starting 2 days ago: []'

Everything worked fine on the first slice, but the second one is empty. What's going on? <br>
<br>
The issue is that list slicing with negative indices can get tricky if we aren't careful. The first time through the loop, we take the slice `daily_balances[-3:-1]` and everything works as expected. However, the second time through the loop, we take the slice daily_balances[-2:0]. <br>
<br>
Our stop index, 0, isn't the *end* of our list, it's the *first* item! So, we just asked for a slice from the next-to-last item to the very first item, which is definitely not what we meant to do. <br>
    *So, why didn't Python return daily_balances in reverse order, from the next-to-last item up through the first item? This isn't what we originally wanted, but wouldn't it make more sense than the empty list we got?* <br>
    *Python is happy to slice lists in reverse order, but wants you to be explicit so it knows unambiguously you want reverse slices. If we wanted a reverse slice, we need to use `daily_balances[-2:0, -1]` where the third parameter, -1, is the step argument, telling Pyhton to reverse the order of the items it slices.* <br>
Essentially, when we asked for the slice `daily_balances[-2:0]` we asked for all the elements starting 2 from the end whose index was less than 0. That's an empty set of numbers, but not an error, so Python returns an empty slice. 

***So how do we fix this code?***<br>
Since daily_balances is just a regular list, the fix is simple-use positive indices instead: 

In [9]:
def show_balances(daily_balances):
    num_balances = len(daily_balances)
    
    #still avoid slice that just has yesterday
    for day in range(num_balances - 3, num_balances -1):
        balance_slice = daily_balances[day : day +2]
        
    #need to calculate how many days ago
    days_ago = num_balances - day
    print("slice starting %d days ago: %s" % (abs(days_ago), balance_slice))

In [10]:
show_balances(daily_balances)

slice starting 2 days ago: [109.86, 110.15]


### Count Capital Letters 
Write a one-liner that will count the number of capital letters in a file. Your code should work even if the file is too big to fit in memory. <br>
Assume you have an open file handle object, such as: <br>

`with open(SOME_LARGE_FILE) as fh: 
count = #your code here`

In [11]:
#to read a text file, use: 
fh = open("hello.txt.", 'r')
print(fh.readlines())

FileNotFoundError: [Errno 2] No such file or directory: 'hello.txt.'

***ANSWER*** <br>
Trying to pull a one-liner out of thin air can be daunting and error-prone. Instead of trying to tackle that head-on, let's work on understanding the framework of our answer, and only afterwared try to ocnvert it to a one-liner. Our first thought might be to keep a running count as we look through the file: <br>


In [12]:
count = 0
text = fh.read()

for character in text: 
    
    if charact.isupper():
        count += 1
        

NameError: name 'fh' is not defined

This is a great start -the code isn't very long, but it is clear. That makes it easier to iterate over as we build up our solution. There are two main issues with what we have so far if we want to turn it into a one-liner: 
- ***Variable Initialization***: in a one-liner, we won't be able to use something like count = 0
- ***Memory***: the question says this needs to work even on files that won't fit into memory, so we can't just read the whole file into a string. <br>
Let's try to deal with the memory issue first: we can't use the read() method since that reads the whole file at once. There are some other common file methods, () and (), that might help so let's look at them. 
- readline() is a method that reads a file into a list- each line is a different item in the list. That doesn't really help us- it's still reading the entire file at once, so we still won't have room. 
- readline() only reads a single line at a time - it seems more promising. This is great from a memory perspective (let's assume each line fits in memory, at least for now). <br>
<br>
The idea of repeatedly calling a function, such as readline(), until we hit some value (the end of the file) is so common, there's a standard library function for it: iter(). We'll need the two-argument form of iter(), where the first argument is our function to call repeatedly, and the second argument is the value that tells us when to stop (also called the *sentinel*) <br>
<br>
What value do we need as our *sentinel* ? Looking at the documentation for readline(), it includes the newline character so even blank lines will have at least one character. It returns an empty string *only* when it hits the end of the file, so our *sentinel* is ''. 

In [13]:
count = 0

for line in iter(fh.readline,''):
    
    if character.isupper():
        count +=1

NameError: name 'fh' is not defined

And this works! But...it's not as clear as it could be. Understanding this code requires knowing about the two-argument iter() and that readline() returns '' at the end of the file. Trying to condense all this into a one-liner seems like it mught be confusing to folloq. <br>
<br>
Is there a simpler way to ierate ovet the lines in the file? If you're using Python 3, there aren't any methods for that on your file handle. If you're using Python 2.7, there *is* something that sounds interesting -`xreadlines()`. It iterates over the lines in a file, yielding each one to let us process it before reading the next line. In our code, it might be used like: <br>

In [14]:
count = 0 

for line in fh.xreadlines():
    
    for character in line: 
        
        if character.isupper(): 
            count += 1

NameError: name 'fh' is not defined

It's exactly like our code with readline() and iter() but even clearer! It's a shame we can't use this in Python3.x though, it seems like it would be great. Let's look at the documentation for this method to see if we can learn what alternatives Python3.x might have: <br>
 `>> help(fh.readlines)`
 `xreadlines() -> reutnrs self`. 
 
 `For backwards compatibility. File objects now include the performance optimizaitons previously implemented in the xreadlines module`

'Return self'? <br> How does that even do anything? <br>
<br>
What's happening here is that iterating over the lines of a file is so common that they built it right in to the object itself. If we use our file object in an irerator, it starts yielding us lines, just like xreadlines()! So we can clean up our code, and make it Python3.x compatible, by just removing xreadlines(). 

In [15]:
count = 0 
for line in fh: 
    
    for character in line: 
        
        if character.isupper(): 
            
            count += 1

NameError: name 'fh' is not defined

Alright, we've finally solved the issue of efficiently reading the file and iterating over it, but we haven't made any progress on making it a one-liner. As we said in the beginning, we can't initialize variables, so what we need is a function that will return the count of all capitalizsed letters. There isn't a count() function in Python (at least, not one that would help us here), but we can rephrase the question just enough to find a function that gets the job done. <br>
<br>
Instead of thinking about a "count of capitalized letters", let's think about mapping every letter (every character, even) to a number, since our answer is a number. All we care about are capital letters, and each one adds exactly 1 to our final count. Every other character should be ignored, or add 0 to our final count. We can get this mapping into a single line using Pyhton's inline if-else: <br>

In [16]:
count = 0

for line in fh: 
    
    for charactier in line: 
        
        count +=(1 if character.isupper() else 0)

NameError: name 'fh' is not defined

What did mapping get us? Well, Python didn't have a function to count capital letters, but it *does* have a function to add up a bunch of 1s and 0s: sum(). <br>
<br>
`sum()` takes any iterable, such as a generator expression, and our latest solution-nested for loops and a single if-else-can easily be rewritten as a generator expression: 


In [17]:
count = sum(1 if character.isupper() else 0 for line in fh for character in line)

NameError: name 'fh' is not defined

and now we've got a one-liner! It's not quite as clear as it could be -seems unnecessary to explicitly sum 0 whenever sum 0 whwnever we have a character that isn't a capital letter. We can filter those out: 

In [18]:
count = sum(1 for line in fh for character in line if character.isupper())

NameError: name 'fh' is not defined

or we can even take advantage of the fact that Python will coerce True to 1 (and False to 0): 

In [19]:
count = sum(character.isupper() for line in fh for character in line)

NameError: name 'fh' is not defined

### Best in Subclass
When I was younger, my parents always said I could have pets if I promised to take care of them. Now that I'm an adult, I decided the best way to keep track of them is with some Python classes!

In [20]:
class Pet(object): 
    num_pets = 0
    
    def __init__(self, name): 
        self.name = name
        self.num_pets += 1
        
        
    def speak(self): 
        print("My name's %s and the number of pets is %d" % (self.name, self.num_pets))

Since these pets won't sit still long enough to be put into a list, I need to keep track with the class attrtibute num_pets.  <br>
<br>
That should be enough to get me started. Let's create a few pets: 

In [21]:
rover = Pet("rover")
spot = ("spot")

and see what they have to say: 

In [22]:
rover.speak()
spot.speak()

My name's rover and the number of pets is 1


AttributeError: 'str' object has no attribute 'speak'

Hmm...I'm not getting the output I expect. What did these two lines print, and how do we fix it?

Something isn't right -it's not counting the number of pets properly. Don't worry Rover, I didn't *replace* you with Spot! What's happening here? <br>
<br>
Turns out, there's the difference between class and instance - attritbues. When we created rover and added to num_pets, we accidentally shadowed Pet.num_pets with rover.num_pets-and they're two completely different variables now! We can see this even more clearly if we ask our Pet class how many pets it know about: 

In [23]:
print(Pet.num_pets)

0


Our Pet class still thinks there are 0 pets, because each new pet adds 1 *and shadows the class attribute num_pets with its own instance attribute*  <br>
So how can we fix this? We just need to make sure we refer to, and increment, the *class* attribute: 

In [24]:
class Pet(object): 
    num_pets = 0
    def __init__(self, name): 
        self.name = name
        
        #change from: self.num_pets += 1
        Pet.num_pets += 1
    def speak(Pet): 
        print("My name's %s and the number of pets is %d" % (Pet, Pet.num_pets))

and now, if we run our updated code: 

In [25]:
rover = Pet("Rover")
spot = Pet("Spot")
rover.speak()
spot.speak()

My name's <__main__.Pet object at 0x7fde44cb9c90> and the number of pets is 2
My name's <__main__.Pet object at 0x7fde44cb9c50> and the number of pets is 2


### When is a number not itself?
Given some simle variables: 


In [26]:
big_num_1 = 1000
big_num_2 = 1000
small_num_1 = 1
small_num_2 = 1

What's the output we get from running the following?

In [27]:
big_num_1 is big_num_2

False

In [28]:
small_num_1 is small_num_2

True

## Answer <br>
The python keyword is tests whether two variables refer to the exact same object, not just if they are equal. For example: 

In [29]:
list_1 = [1,2,3]
list_2 = [1,2,3]
print("list_1 == list_2? %s" % (list_1 == list_2))
print("list_1 is list_2? %s" % (list_1 is list_2))

list_1 == list_2? True
list_1 is list_2? False


Now let's look at our original questions: 

In [30]:
big_num_1 is big_num_2

False

Suppose to return 
False

This hsould make sense- we created two different objects that each hold a number, so while they happen to hold the same value, they aren't referring to the same object in memory. <br>
And for small numbers?

In [31]:
small_num_1 is small_num_2

True

What's happening here? **Python creates singletons for the most commonly used integars**. One good reason to do this is that small numbers get used so frequently that if Python had to create a brand new object every time it needed a number, and then free the object when it goes out of scope, it would start to actually take a noticeable amount of time

                    **The idea behind singletons is that there can only ever be one instance of a particular object, and whenever someone tries to use or create a new one, they get the original.**

So, which numbers count as 'small numbers'? We can write a quick bit of code to test this out for us: 

In [32]:
#Python ranges start at 0 and don't include the last number
for num in range(1001):
    #try to create a new object
    num_copy = num * 1
    
    if num is num_copy: 
        print(num, 'is a singleton')
    else: print(num, 'is not a singleton')

0 is a singleton
1 is a singleton
2 is a singleton
3 is a singleton
4 is a singleton
5 is a singleton
6 is a singleton
7 is a singleton
8 is a singleton
9 is a singleton
10 is a singleton
11 is a singleton
12 is a singleton
13 is a singleton
14 is a singleton
15 is a singleton
16 is a singleton
17 is a singleton
18 is a singleton
19 is a singleton
20 is a singleton
21 is a singleton
22 is a singleton
23 is a singleton
24 is a singleton
25 is a singleton
26 is a singleton
27 is a singleton
28 is a singleton
29 is a singleton
30 is a singleton
31 is a singleton
32 is a singleton
33 is a singleton
34 is a singleton
35 is a singleton
36 is a singleton
37 is a singleton
38 is a singleton
39 is a singleton
40 is a singleton
41 is a singleton
42 is a singleton
43 is a singleton
44 is a singleton
45 is a singleton
46 is a singleton
47 is a singleton
48 is a singleton
49 is a singleton
50 is a singleton
51 is a singleton
52 is a singleton
53 is a singleton
54 is a singleton
55 is a singleton
56

626 is not a singleton
627 is not a singleton
628 is not a singleton
629 is not a singleton
630 is not a singleton
631 is not a singleton
632 is not a singleton
633 is not a singleton
634 is not a singleton
635 is not a singleton
636 is not a singleton
637 is not a singleton
638 is not a singleton
639 is not a singleton
640 is not a singleton
641 is not a singleton
642 is not a singleton
643 is not a singleton
644 is not a singleton
645 is not a singleton
646 is not a singleton
647 is not a singleton
648 is not a singleton
649 is not a singleton
650 is not a singleton
651 is not a singleton
652 is not a singleton
653 is not a singleton
654 is not a singleton
655 is not a singleton
656 is not a singleton
657 is not a singleton
658 is not a singleton
659 is not a singleton
660 is not a singleton
661 is not a singleton
662 is not a singleton
663 is not a singleton
664 is not a singleton
665 is not a singleton
666 is not a singleton
667 is not a singleton
668 is not a singleton
669 is not 

Python makes singletons for the numbers 0 through 256. But are there other numbers made into singeltons? It would make sense that some negative numbers might be worth making only once - it's pretty common to look at, say, the last few characters in a string, or the last few elements in a list. We can run the same code but check negative numbers instead: 

In [33]:
#up to but not including 0 - we already checked it
for num in range(-1000, 0): 
    
   # try to create a new object
    num_copy = num*1 
    
    if num is num_copy: 
        print (num, 'is a singleton')
    else: 
        print(num, 'is not a singleton')

-1000 is not a singleton
-999 is not a singleton
-998 is not a singleton
-997 is not a singleton
-996 is not a singleton
-995 is not a singleton
-994 is not a singleton
-993 is not a singleton
-992 is not a singleton
-991 is not a singleton
-990 is not a singleton
-989 is not a singleton
-988 is not a singleton
-987 is not a singleton
-986 is not a singleton
-985 is not a singleton
-984 is not a singleton
-983 is not a singleton
-982 is not a singleton
-981 is not a singleton
-980 is not a singleton
-979 is not a singleton
-978 is not a singleton
-977 is not a singleton
-976 is not a singleton
-975 is not a singleton
-974 is not a singleton
-973 is not a singleton
-972 is not a singleton
-971 is not a singleton
-970 is not a singleton
-969 is not a singleton
-968 is not a singleton
-967 is not a singleton
-966 is not a singleton
-965 is not a singleton
-964 is not a singleton
-963 is not a singleton
-962 is not a singleton
-961 is not a singleton
-960 is not a singleton
-959 is not a s

-629 is not a singleton
-628 is not a singleton
-627 is not a singleton
-626 is not a singleton
-625 is not a singleton
-624 is not a singleton
-623 is not a singleton
-622 is not a singleton
-621 is not a singleton
-620 is not a singleton
-619 is not a singleton
-618 is not a singleton
-617 is not a singleton
-616 is not a singleton
-615 is not a singleton
-614 is not a singleton
-613 is not a singleton
-612 is not a singleton
-611 is not a singleton
-610 is not a singleton
-609 is not a singleton
-608 is not a singleton
-607 is not a singleton
-606 is not a singleton
-605 is not a singleton
-604 is not a singleton
-603 is not a singleton
-602 is not a singleton
-601 is not a singleton
-600 is not a singleton
-599 is not a singleton
-598 is not a singleton
-597 is not a singleton
-596 is not a singleton
-595 is not a singleton
-594 is not a singleton
-593 is not a singleton
-592 is not a singleton
-591 is not a singleton
-590 is not a singleton
-589 is not a singleton
-588 is not a si

-129 is not a singleton
-128 is not a singleton
-127 is not a singleton
-126 is not a singleton
-125 is not a singleton
-124 is not a singleton
-123 is not a singleton
-122 is not a singleton
-121 is not a singleton
-120 is not a singleton
-119 is not a singleton
-118 is not a singleton
-117 is not a singleton
-116 is not a singleton
-115 is not a singleton
-114 is not a singleton
-113 is not a singleton
-112 is not a singleton
-111 is not a singleton
-110 is not a singleton
-109 is not a singleton
-108 is not a singleton
-107 is not a singleton
-106 is not a singleton
-105 is not a singleton
-104 is not a singleton
-103 is not a singleton
-102 is not a singleton
-101 is not a singleton
-100 is not a singleton
-99 is not a singleton
-98 is not a singleton
-97 is not a singleton
-96 is not a singleton
-95 is not a singleton
-94 is not a singleton
-93 is not a singleton
-92 is not a singleton
-91 is not a singleton
-90 is not a singleton
-89 is not a singleton
-88 is not a singleton
-87 

This shows that numbers from -5 up to and including 256 have singleton instances, so they could be tested against each other with *is*. But we shouldn't do that, since the enxt version of Python might change the range of singleton numbers. And any code that took advantage of these singletons won't be able to check for equality for numbers outside this range. A simple equality check with numbers is always safest. 

# Generating the Matrix
We want